In [9]:
import zarr_fuse as zf
import polars as pl
import pandas as pd
import numpy as np

# Import Plotly libraries
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import custom Plotly visualization classes
try:
    # Try importing if files are in the same directory
    from plot_map_view import InteractiveMapPlotter
    from plot_time_zoom import MultiZoomer
except ImportError:
    # Fallback: Import from the zarr_fuse.plot package structure
    from zarr_fuse.plot.plot_map_view import InteractiveMapPlotter
    from zarr_fuse.plot.plot_time_zoom import MultiZoomer

In [10]:
from hlavo_surface import inputs
work_dir = inputs.work_dir


#root_node = zf.Node.read_store(work_dir / "surface.zarr")

from dotenv import load_dotenv
from pathlib import Path
path = Path("../secrets.env").absolute()
print("Loading env file: ", path)
load_dotenv(path)  # or Path("secrets.env")

root_node = zf.open_store(inputs.surface_schema_yaml, workdir=work_dir)

# Configuration of the polts which variables/fields to show.
fields = {
    'yr.no': ['air_temperature', 'precipitation_amount', 'relative_humidity']
}


#df_temp = root_node['yr.no'].read_df("air_temperature", date_time=slice(None))
#print(df_temp)
print(root_node['yr.no'].dataset.coords['date_time'])

Loading env file:  C:\Users\fatih\Documents\GitHub\zarr_fuse\app\hlavo_surface\notebooks\..\secrets.env
DEBUG: Entering Node.dataset
DEBUG: Opening Zarr group at path: 
DEBUG: Dataset loaded. Variables: []
DEBUG: Coordinates: []
DEBUG: Entering Node.dataset
DEBUG: Opening Zarr group at path: yr.no
DEBUG: Dataset loaded. Variables: ['air_temperature', 'cloud_fraction', 'grid_domain', 'latitude', 'dew_point_temp', 'cloud_fraction_low', 'air_pressure', 'cloud_fraction_medium', 'cloud_fraction_high', 'longitude', 'fog_fraction', 'precipitation_amount', 'relative_humidity', 'wind_from_direction', 'wind_speed', 'ultraviolet_index_clear_sky']
DEBUG: Coordinates: ['lat_lon', 'date_time']
DEBUG: Coord lat_lon attrs: {'composed': ['latitude', 'longitude'], 'description': '\n\nExample of coordinate indexed uniquely by a tuple of values. The source columns are converted to the 1D list of tuples.', 'chunk_size': 64}
DEBUG: Coord date_time attrs: {'composed': ['date_time'], 'description': '\n\nNone'

In [11]:

node_path = 'yr.no'
node = root_node[node_path]
ds = node.dataset


overview = zf.plot.build_overview(root_node)
full_coords = overview[node_path].cumul_coordinates


mid_date_time = full_coords['date_time'].quantile(
            0.5,                     # 50 % quantile = median
            method="nearest"         # or "linear", "midpoint", …
).item() 

mid_lat_lon = full_coords['lat_lon'][0]
point_ds = ds.sel(lat_lon=mid_lat_lon)

# Initialize handlers list for synchronization
handlers = []

# Define data selector configuration
# Note: Matplotlib colormaps (plt.cm...) have been removed as they are not needed for Plotly.
data_selector = {
    'date_time': 'time_axis',
    'longitude': 'lon_axis',
    'latitude': 'lat_axis',
    # 'air_temperature': ... (Color mapping is now handled internally or by defaults)
    
    # State variables for interactivity
    'time_point': None,
    'lon_point': None,
    'lat_point': None
}

# Load data into a DataFrame
# slice(None) loads all time steps.
df = node.read_df(fields[node_path], date_time=slice(None))
print(f"Data loaded, row count: {len(df)}")

# Initialize the new Plotly-based Map Plotter
# We use the class imported in Block 1 instead of 'zf.plot.InteractiveMapPlotter'
plotter = InteractiveMapPlotter(df, data_selector, handlers)

# Display the interactive map widget
from IPython.display import display
display(plotter.fig)

DEBUG: Entering Node.dataset
DEBUG: Opening Zarr group at path: yr.no
DEBUG: Dataset loaded. Variables: ['cloud_fraction_high', 'cloud_fraction', 'cloud_fraction_low', 'air_temperature', 'fog_fraction', 'grid_domain', 'longitude', 'relative_humidity', 'dew_point_temp', 'air_pressure', 'wind_from_direction', 'wind_speed', 'cloud_fraction_medium', 'latitude', 'precipitation_amount', 'ultraviolet_index_clear_sky']
DEBUG: Coordinates: ['date_time', 'lat_lon']
DEBUG: Coord date_time attrs: {'composed': ['date_time'], 'description': '\n\nNone', 'chunk_size': 1024}
DEBUG: Coord lat_lon attrs: {'composed': ['latitude', 'longitude'], 'description': '\n\nExample of coordinate indexed uniquely by a tuple of values. The source columns are converted to the 1D list of tuples.', 'chunk_size': 64}
DEBUG: Entering Node.dataset
DEBUG: Opening Zarr group at path: 
DEBUG: Dataset loaded. Variables: []
DEBUG: Coordinates: []
DEBUG: Entering Node.dataset
DEBUG: Opening Zarr group at path: yr.no
DEBUG: Datas

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'lat': {'bdata': ('uh3QiIhuSUCPwvUoXG9JQJc40TJ7bk' ... '0K16NwSUDsUbgehWtJQEkZzLrabklA'),
                      'dtype': 'f8'},
              'lon': {'bdata': ('I5QInozHLUDqUbgehestQHJleeyhyi' ... '6F61G4LUBH4XoUrsctQEXv+QbLyS1A'),
                      'dtype': 'f8'},
              'marker': {'color': [blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
  

In [12]:
# Initialize the Time Series (MultiZoomer) visualization
# We use the custom class imported in Block 1, replacing the old zf.plot version.
zoomer = MultiZoomer(df, data_selector, handlers)

# Display the interactive Plotly widgets
from IPython.display import display
print("Time Series View (Year - Month - Day):")
display(zoomer.fig)

--- DEBUG: Time Series Initialized (Wide View) ---
Time Series View (Year - Month - Day):


FigureWidget({
    'data': [{'fill': 'none',
              'legendgroup': 'air_temperature',
              'line': {'color': '#d62728', 'width': 1.5},
              'marker': {'color': '#d62728', 'size': 0},
              'mode': 'lines',
              'name': 'Air Temp (°C)',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e69bf84c-6941-46a5-a18d-284baa728025',
              'x': array(['2025-09-02T11:00:00.000000000', '2025-09-02T11:00:00.000000000',
                          '2025-09-02T11:00:00.000000000', ..., '2025-12-03T00:00:00.000000000',
                          '2025-12-03T00:00:00.000000000', '2025-12-03T00:00:00.000000000'],
                         shape=(1672,), dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('zczMzMzML0DNzMzMzMwvQM3MzMzMzC' ... 'AAAAD4fwAAAAAAAPh/AAAAAAAA+H8='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'fill': 'none',
              'legendg